In [1]:
import shapefile as shp
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import matplotlib.pyplot as plt

In [115]:
# Load the shapefiles
sf = shp.Reader(r'C:\Users\willd\Documents\Georgia Tech\CSE6424\Project\census_tract_Georgia\cb_2018_13_tract_500k.shp')

In [116]:
records = sf.shapeRecords()

In [117]:
# See whats in the records
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['STATEFP', 'C', 2, 0],
 ['COUNTYFP', 'C', 3, 0],
 ['TRACTCE', 'C', 6, 0],
 ['AFFGEOID', 'C', 20, 0],
 ['GEOID', 'C', 11, 0],
 ['NAME', 'C', 100, 0],
 ['LSAD', 'C', 2, 0],
 ['ALAND', 'N', 14, 0],
 ['AWATER', 'N', 14, 0]]

In [118]:
# Load the tract or ZCTA file
tract_file = 'atlanta_tract_health'
df = pd.read_csv(r'C:\Users\willd\Documents\Georgia Tech\CSE6424\Project\cse6242_group_project\notebooks_updates\data\tract_health\{}.csv'.format(tract_file))
geojsons = []
used_tract_ids = set()

In [119]:
# Match up the records by ID if possible
for record in tqdm(records):
    tract_id = int(record.record[4])
    try:
        row = df.loc[df['TractFIPS'] == tract_id].iloc[0]
    except IndexError:
        continue
    try:
        coords = np.array(record.__geo_interface__['geometry']['coordinates'][0])
        if (coords[:, 0].ptp() > 0.1) or (coords[:, 1].ptp() > 0.1):
            continue
    except Exception:
        pass
    used_tract_ids.add(tract_id)
    d = record.__geo_interface__
    d['properties'] = {'id': tract_id}
    for c in df.columns:
        if c.endswith('_CrudePrev'):
            d['properties'][c] = row[c]
    for c in ('education_closest_travel_times', 'food_closest_travel_times', 'worship_closest_travel_times',
              'physical_closest_dist', 'transport_closest_dist', 'PovertyRate'):
        d['properties']['MedianFamilyIncome' if c == 'Household median income' else c] = row[c]
    geojsons.append(d)

100%|████████████████████████████████████████████████████████████████████████████| 1966/1966 [00:01<00:00, 1687.45it/s]


In [120]:
# Repeat if there are multiple shapefiles (for instance if the city is in two states)
#sf2 = shp.Reader(r'C:\Users\willd\Documents\Georgia Tech\CSE6424\Project\census_tract_New_York\cb_2018_36_tract_500k.shp')

In [121]:
#records2 = sf2.shapeRecords()

In [122]:
# for record in tqdm(records2):
#     tract_id = int(record.record[4])
#     if tract_id in used_tract_ids:
#         continue
#     try:
#         row = df.loc[df['TractFIPS'] == tract_id].iloc[0]
#     except IndexError:
#         continue
#     used_tract_ids.add(tract_id)
#     d = record.__geo_interface__
#     d['properties'] = {'id': tract_id}
#     for c in df.columns:
#         if c.endswith('_CrudePrev'):
#             d['properties'][c] = row[c]
#     for c in ('education_closest_travel_times', 'food_closest_travel_times', 'worship_closest_travel_times',
#               'physical_closest_dist', 'transport_closest_dist', 'LowIncomeTracts', 'PovertyRate', 'MedianFamilyIncome'):
#         d['properties'][c] = row[c]
#     geojsons.append(d)

In [123]:
# How many ZCTAs did we find?
len(geojsons)

196

In [124]:
# Prep the JSON format
geojsons = {"type":"FeatureCollection", "features": geojsons}

In [125]:
# See https://stackoverflow.com/questions/50916422/python-typeerror-object-of-type-int64-is-not-json-serializable
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            if np.isnan(obj):
                return None
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, float) and np.isnan(obj):
            return None
        return super(NpEncoder, self).default(obj)

In [126]:
# Save if off based on the original name
with open(r'C:\Users\willd\Documents\Georgia Tech\CSE6424\Project\cse6242_group_project\visualization\{}.json'.format(tract_file), 'w') as f:
    json.dump(geojsons, f, cls=NpEncoder)